In [25]:
import random
import sys
from math import sqrt
import numpy as np
import pygame
from pygame import RESIZABLE, QUIT, KEYUP, K_ESCAPE

In [26]:
radius = 50
screen_width = 1280
screen_height = 720

#### Расстояние между точками

In [27]:
def distance(a, b):
    return sqrt((a[0] - b[0]) ** 2 + (a[1] - b[1]) ** 2)

#### Генерация рандомных данных

In [28]:
def generate_data(number_of_points, number_of_class):
    data = []
    for classNum in range(number_of_class):
        center_x, center_y = random.randint(radius, screen_width - radius), random.randint(radius, screen_height - radius)
        for rowNum in range(number_of_points):
            data.append([[random.gauss(center_x, radius / 2), random.gauss(center_y, radius / 2)], classNum])
    return data

#### Поиск самого популярного цвета

In [29]:
def find_color(points):
    color_counts = {}
    for p in points:
        color_counts.setdefault(p[1], 0)
        color_counts[p[1]] += 1
    popular_color = 0
    counter = 0
    for color, color_counter in color_counts.items():
        if color_counter > counter:
            counter = color_counter
            popular_color = color
    return popular_color

#### Поиск расстояния между точками

In [30]:
def find_distances_between_points(point):
    points_distances = []
    for p in points:
        if point == p:
            continue
        points_distances.append(distance(p[0], point[0]))
    return [points[x] for x in np.argsort(points_distances)]

#### Пересчет значения k

In [31]:
def recount_k(point):
    distances = find_distances_between_points(point)
    for k in range(1, len(k_results)):
        nearest_points = distances[:k]
        current_result = find_color(nearest_points)
        if point[1] == current_result:
            k_results[k] += 1

#### Расчета значения k

In [32]:
def calculate_k():
    result = 0
    index = -1
    for k_index, k in enumerate(k_results):
        if k > result:
            result = k
            index = k_index
    return index

#### Запуск алгоритма KNN

In [33]:
def knn(new_point, k):
    sorted_points = find_distances_between_points(new_point)
    nearest_k_points = sorted_points[:k]
    color = find_color(nearest_k_points)
    new_point[1] = color

number_of_points = 10
number_of_class = 3
points = generate_start_points(number_of_points, number_of_class)
new_points = []
global_index = 0

k_results = [0 for x in range(int(len(points)))]

surface = pygame.display.set_mode((screen_width, screen_height), RESIZABLE)
surface.fill((255, 255, 255))

colors = ['red', 'green', 'blue', 'black']


while True:
    for event in pygame.event.get():
        if event.type == QUIT or (event.type == KEYUP and event.key == K_ESCAPE):
            pygame.quit()
            sys.exit()
        if event.type == pygame.MOUSEBUTTONDOWN:
            if event.button == 1:
                new_points.append([[event.pos[0], event.pos[1]], -1])

            if event.button == 3:
                point = [[event.pos[0], event.pos[1]], -1]
                knn(point, calculate_k())
                points.append(point)

        if event.type == pygame.KEYDOWN:
            if len(new_points) > global_index:
                current_point = new_points[global_index]
                if event.key == pygame.K_r:
                    current_point[-1] = 0
                    global_index += 1
                    points.append(current_point)
                    recount_k(current_point)
                if event.key == pygame.K_g:
                    current_point[-1] = 1
                    global_index += 1
                    points.append(current_point)
                    recount_k(current_point)
                if event.key == pygame.K_b:
                    current_point[-1] = 2
                    global_index += 1
                    points.append(current_point)
                    recount_k(current_point)
    for point in points:
        pygame.draw.circle(surface, colors[point[1]], point[0], 5)
    for point in new_points:
        pygame.draw.circle(surface, colors[point[1]], point[0], 5)
    pygame.display.update()

SystemExit: 